In [12]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine-date')

data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine[['class']].to_numpy()

from sklearn.model_selection import train_test_split as tts
train_input,test_input,train_target,test_target = \
tts(data, target, test_size=0.2, random_state=42)
sub_input, val_input, sub_target, val_target = \
tts(train_input, train_target, test_size= 0.2, random_state = 42)

In [13]:
print(sub_input.shape, val_input.shape, test_input.shape)

(4157, 3) (1040, 3) (1300, 3)


In [14]:
from sklearn.tree import DecisionTreeClassifier as dtc
dt = dtc(random_state = 42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [15]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00995946, 0.00710607, 0.00750256, 0.00737071, 0.00709891]), 'score_time': array([0.00079226, 0.0007031 , 0.00068474, 0.00076962, 0.00068569]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [16]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [17]:
from sklearn.model_selection import StratifiedKFold as sfk
scores = cross_validate(dt, train_input, train_target, cv=sfk())
print(np.mean(scores['test_score']))
splitter = sfk(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.855300214703487
0.8574181117533719


In [20]:
#연습시작
from sklearn.model_selection import GridSearchCV as gsc
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = gsc(dtc(random_state = 42), params,n_jobs=-1)

In [21]:
gs.fit(train_input, train_target)
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [22]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [23]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [24]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])
#여기까지 연습

{'min_impurity_decrease': 0.0001}


In [25]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5,20,1),
          'min_samples_split' : range(2,100,10)}

In [28]:
gs = gsc(dtc(random_state=44), params, n_jobs= -1)
gs.fit(train_input, train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=44,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'm

In [29]:
print(gs.best_params_)

{'max_depth': 19, 'min_impurity_decrease': 0.0001, 'min_samples_split': 2}


In [30]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8691541422965869


In [31]:
from scipy.stats import uniform , randint
rgen= randint(0,10)
rgen.rvs(10)

array([2, 4, 3, 5, 6, 2, 1, 9, 7, 6])

In [32]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 99,  92, 101,  98,  81, 111,  99, 108, 116,  95]))

In [36]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20,50),
          'min_samples_split' : randint(2,25),
          'min_samples_leaf':randint(1,25)
          }

In [37]:
from sklearn.model_selection import RandomizedSearchCV as rsc
gs = rsc(dtc(), params, n_iter=100,n_jobs=-1)
gs.fit(train_input, train_target)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=None,
       

In [38]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))
dt=gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 48, 'min_impurity_decrease': 0.00035266878397853887, 'min_samples_leaf': 5, 'min_samples_split': 3}
0.8695426445546754
0.8576923076923076
